In [ ]:
### Predict Messages
from langchain.chat_models import ChatOpenAI
from langchain.schema  import HumanMessage, AIMessage, SystemMessage
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]) ## call Chat large language models API.

messages  = [
    SystemMessage(content='you are a geography expert. and you only reply in korean.'),
    AIMessage(content='무엇이 궁금하신가요? 저는 지리학 박사입니다!'),
    HumanMessage(content='한국과 일본의 지리적 차이에 대해 알려줘.')
]

# Pass a message sequence to the model and return a message prediction.
chat.predict_messages(messages)

In [ ]:
### Prompt Templates
from langchain.prompts import PromptTemplate, ChatPromptTemplate 

# Load a prompt template from a template.
template = PromptTemplate.from_template('{country_a}과 {country_b}의 지리적 차이에 대해 알려줘.')

prompt = template.format(
country_a='korea',
country_b='japan')

# Pass a single string input to the model and return a string prediction.
# chat.predict(prompt) 

# Create a chat prompt template from a variety of message format그냥,, s.
template = ChatPromptTemplate.from_messages ([
    ('system', 'you are a geography expert. and you only reply in {language}.'),
    ('ai', '무엇이 궁금하신가요? 저는 {name}입니다!'),
    ('human', '{country_a}과 {country_b}의 지리적 차이에 대해 알려줘.')
])

prompt = template.format_messages(language="korean", name='배다롱', country_a='한국', country_b='일본')

# Pass a message sequence to the model and return a message prediction.
# chat.predict_messages(prompt) 

In [ ]:
from langchain.schema import BaseOutputParser

# Output parsers help structure language model responses.
# My personal output parser
class CommaOutputParser(BaseOutputParser):
    # Parse a single string model output into some structure.
    def parse(self,text): 
        items= text.strip().split(',')
        return list(map(str.strip, items))
    
    
p = CommaOutputParser()
# p.parse("hello, how, are, you?")    
# ['hello', 'how', 'are', 'you?']

In [ ]:

template = ChatPromptTemplate.from_messages([
    ("system", "you are a list generating machine. everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. do not reply with anything else."),
    ("human", "{question}"),
])

# general way
# prompt = template.format_messages(max_items=10, question = 'what are the colors?')
# results = chat.predict_messages(prompt)
# p = CommaOutputParser()
# p.parse(results.content)

# LangChain Expression Language, or LCEL, is a declarative way to easily compose chains together. 
# RunnableSequence을 사용하여 여러 Runnable을 조합하고 체인을 구성할 수 있습니다. 
chain = template | chat | CommaOutputParser()
# chain.invoke({"max_items":5, "question": "flowers"})

In [ ]:
# Chaining Chains
chef_template = ChatPromptTemplate.from_messages([
    ("system", "you are a world-class international chef. you create easy to follow recipies for any typeof cuisins with easy to find ingredients."),
    ("human", "I want to cook {cuision} food."),
])

chef_chain = chef_template | chat 

In [ ]:
veg_chef_template = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}"),
])

veg_chain = veg_chef_template | chat

In [ ]:
final_chain = {"recipe": chef_chain} | veg_chain
# final_chain.invoke({'cuision':'indian'})

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts import load_prompt

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])  

examples = [
    {
    "question": "What do you know about France?",
    "answer": """
    Here is what I know:
    Capital: Paris
    Language: French
    Food: Wine and Cheese
    Currency: Euro
    """,
    },
    {
    "question": "What do you know about Italy?",
    "answer": """
    I know this:
    Capital: Rome
    Language: Italian
    Food: Pizza and Pasta
    Currency: Euro
    """,
    },
    {
    "question": "What do you know about Greece?",
    "answer": """
    I know this:
    Capital: Athens
    Language: Greek
    Food: Souvlaki and Feta Cheese
    Currency: Euro
    """,
    }
]

examples_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(examples_template)

# Select examples based on length.
example_selector_length = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=80
)


# BaseExampleSelector: Interface for selecting examples to include in prompts.
# RandomExampleSelector: 여러 examples 중 랜덤으로 하나 뽑는 my 함수
class RandomExampleSelector(BaseExampleSelector):
    
    def __init__ (self, examples): 
        self.examples = examples 

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]


# Interface for selecting examples to include in prompts.
example_selector_random = RandomExampleSelector(
    examples=examples,
)

# Prompt template that contains few shot examples.
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector_random,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

# prompt.format(country="Brazil")

In [ ]:
examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

# Chat prompt template that supports few-shot examples.
example_few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_few_shot_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

# chain.invoke({"country": "Thailand"})

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

# seperated prompt - loading a prompt from LangChainHub or local fs. (json file, yaml file)
prompt = load_prompt('./prompt.yaml')
# prompt.format(country="China")  

intro = PromptTemplate.from_template(
"""
You are a role playing assistant.
And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
"""
This is an example of how you talk:

Human: {example_question}
You: {example_answer}
"""
)

start = PromptTemplate.from_template(
"""
Start now!

Human: {question}
You:
"""
)

final = PromptTemplate.from_template(
"""
{intro}
{example}
{start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

# PipelinePromptTemplate: prompt template for composing multiple prompt templates together.
# 리액트 컴토넌트처럼 프롬프트들을 재사용할 때 유용하다. 하나의 메인 프롬프트(final_prompt)가 존재하고, 나머지 자잘한 프롬프트들을 파이프 라인으로 이어준다.
full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)

chain = full_prompt | chat
chain.invoke({
    "character":"Cat", 
    "example_question":"what is your location?", 
    "example_answer":"Meow Meow! that is secret! Meow Meow!", 
    "question":'what is your favorite food?'
}).content



Meow Meow! I love fish! Meow Meow!

'Meow Meow! I love fish! Meow Meow!'